# Investors' Sentiment & S&P500 : Merge Dataset
---

## Notebook Organisation:
1. Data Collection - Subreddit
2. Data Collection - Target 
3. **Merging Data (Current Notebook)**
2. EDA and Preprocessing
3. Model Tuning and Insights

## Import Library
---

In [129]:
import requests
import time
import nltk
import pandas as pd
import regex as re
import numpy as np
import random
import seaborn as sns
import scipy.stats as stats
import warnings
import string
from datetime import datetime
import datetime as dt

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, TweetTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')
sns.set_style('ticks')
%matplotlib inline

## Read CSV
---

In [130]:
df = pd.read_csv("data/final_df.csv")
target = pd.read_csv("data/sp500_diff.csv")

### Reddit Dataset

In [131]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['Date'] = df['date'].dt.date
df.loc[:, 'year_and_month'] = df.loc[:, 'date'].apply(lambda x: '{}-{:02d}'.format(x.year, x.month))
df.loc[:, 'week_of_year'] = df.loc[:, 'date'].dt.isocalendar().week

In [132]:
df.head()

,date,title,selftext,is_self,upvotes,n_comments,permalink,author,month,year,Date,year_and_month,week_of_year
0,2019-01-03 18:02:40,Backtesting moving average crossover,"Hello guys, \nI was trying to backtest a movi...",1.0,1.0,0.0,/r/stocks/comments/ac4c0u/backtesting_moving_a...,bhandarimohit2029,1,2019,2019-01-03,2019-01,1
1,2019-01-03 20:07:03,"r/Stocks Daily Discussion Thursday - Jan 03, 2019",These daily discussions run from Monday to Fri...,1.0,1.0,12.0,/r/stocks/comments/ac54dw/rstocks_daily_discus...,AutoModerator,1,2019,2019-01-03,2019-01,1
2,2019-01-03 20:12:03,Can someone ELI5 what a cross signal Index is ...,I just finished Margin call on Netflix and was...,1.0,1.0,0.0,/r/stocks/comments/ac55o3/can_someone_eli5_wha...,tellmetheworld,1,2019,2019-01-03,2019-01,1
3,2019-01-03 20:54:22,Your AM Global Stocks Preview and a whole lot ...,\n\n### US Stocks\n\n* **US stocks index futu...,1.0,1.0,16.0,/r/stocks/comments/ac5gf6/your_am_global_stock...,QuantalyticsResearch,1,2019,2019-01-03,2019-01,1
4,2019-01-03 21:01:23,Gld outperforms snp and Dow,Gld has outperformed the snp and Dow since 200...,1.0,1.0,30.0,/r/stocks/comments/ac5ibv/gld_outperforms_snp_...,1anon2y3mous,1,2019,2019-01-03,2019-01,1


In [133]:
df.tail(3)

,date,title,selftext,is_self,upvotes,n_comments,permalink,author,month,year,Date,year_and_month,week_of_year
508251,2020-12-31 23:46:51,Daily candles vs 8hr candles,Is the daily candle the result of 3 trading se...,1.0,1.0,2.0,/r/technicalanalysis/comments/kns8ps/daily_can...,engineertee,12,2020,2020-12-31,2020-12,53
508252,2021-01-01 02:41:41,"Weekly Technical Forecast 31st December, 2020 ...",NaN,0.0,1.0,0.0,/r/technicalanalysis/comments/knvhz8/weekly_te...,ResearchSquared,1,2021,2021-01-01,2021-01,53
508253,2021-01-01 07:20:24,Quick Question - How do you decide which chart...,Guess the question is in the title...but do yo...,1.0,1.0,5.0,/r/technicalanalysis/comments/ko0dho/quick_que...,JustArran12345,1,2021,2021-01-01,2021-01,53


In [134]:
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508254 entries, 0 to 508253
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            508254 non-null  datetime64[ns]
 1   title           508254 non-null  object        
 2   selftext        349645 non-null  object        
 3   is_self         508254 non-null  float64       
 4   upvotes         508254 non-null  float64       
 5   n_comments      508254 non-null  float64       
 6   permalink       508254 non-null  object        
 7   author          508254 non-null  object        
 8   month           508254 non-null  int64         
 9   year            508254 non-null  int64         
 10  Date            508254 non-null  datetime64[ns]
 11  year_and_month  508254 non-null  object        
 12  week_of_year    508254 non-null  UInt32        
dtypes: UInt32(1), datetime64[ns](2), float64(3), int64(2), object(5)
memory usage: 49.0+ MB


### Target Dataset

In [135]:
target.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume,month,year,Diff,Percent_Change,Percent_Change_Class
0,2019-01-02,2476.959961,2519.489990,2467.469971,2510.030029,2510.03,3733160000,1,2019,NaN,NaN,NaN
1,2019-01-03,2491.919922,2493.139893,2443.959961,2447.889893,2447.89,3822860000,1,2019,-62.140137,-0.025385,down
2,2019-01-04,2474.330078,2538.070068,2474.330078,2531.939941,2531.94,4213410000,1,2019,84.050050,0.033196,up


In [136]:
target['Percent_Change_Class'] = target['Percent_Change_Class'].shift(-1)
target.dropna(inplace = True)
target['Date'] = pd.to_datetime(target['Date'])

In [137]:
target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 1 to 503
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  503 non-null    datetime64[ns]
 1   Open                  503 non-null    float64       
 2   High                  503 non-null    float64       
 3   Low                   503 non-null    float64       
 4   Close                 503 non-null    float64       
 5   Adj Close             503 non-null    float64       
 6   Volume                503 non-null    int64         
 7   month                 503 non-null    int64         
 8   year                  503 non-null    int64         
 9   Diff                  503 non-null    float64       
 10  Percent_Change        503 non-null    float64       
 11  Percent_Change_Class  503 non-null    object        
dtypes: datetime64[ns](1), float64(7), int64(3), object(1)
memory usage: 51.1+ KB


## Merging Dataset with Target Variable
---

### Function to dataset with target variable based on date

In [213]:
# Matching the percentage_change_class based on date between the feature_df and target_df
def match_dates_and_pull_y(features_df, target_df):
    change = []

    for i in features_df['Date']:
        try:
            if i in list(target_df['Date']):
                change.append(target_df['Percent_Change_Class'].loc[target_df['Date']==i].values)
            elif i not in list(target_df['Date']):
                change.append('x')
                pass
        except Exception as e:
            print('Error:', e)
            
    return pd.DataFrame(change)

### Merge dataset

In [139]:
raw_combined_df = df.merge(change_df, left_index=True, right_index=True)

In [140]:
raw_combined_df.rename(columns={0:'percent_change_class'}, inplace=True)
raw_combined_df.head()

,date,title,selftext,is_self,upvotes,n_comments,permalink,author,month,year,Date,year_and_month,week_of_year,percent_change_class
0,2019-01-03 18:02:40,Backtesting moving average crossover,"Hello guys, \nI was trying to backtest a movi...",1.0,1.0,0.0,/r/stocks/comments/ac4c0u/backtesting_moving_a...,bhandarimohit2029,1,2019,2019-01-03,2019-01,1,up
1,2019-01-03 20:07:03,"r/Stocks Daily Discussion Thursday - Jan 03, 2019",These daily discussions run from Monday to Fri...,1.0,1.0,12.0,/r/stocks/comments/ac54dw/rstocks_daily_discus...,AutoModerator,1,2019,2019-01-03,2019-01,1,up
2,2019-01-03 20:12:03,Can someone ELI5 what a cross signal Index is ...,I just finished Margin call on Netflix and was...,1.0,1.0,0.0,/r/stocks/comments/ac55o3/can_someone_eli5_wha...,tellmetheworld,1,2019,2019-01-03,2019-01,1,up
3,2019-01-03 20:54:22,Your AM Global Stocks Preview and a whole lot ...,\n\n### US Stocks\n\n* **US stocks index futu...,1.0,1.0,16.0,/r/stocks/comments/ac5gf6/your_am_global_stock...,QuantalyticsResearch,1,2019,2019-01-03,2019-01,1,up
4,2019-01-03 21:01:23,Gld outperforms snp and Dow,Gld has outperformed the snp and Dow since 200...,1.0,1.0,30.0,/r/stocks/comments/ac5ibv/gld_outperforms_snp_...,1anon2y3mous,1,2019,2019-01-03,2019-01,1,up


In [141]:
raw_combined_df['percent_change_class'].unique()

array(['up', 'x', 'down'], dtype=object)

In [142]:
combined_df_x_only = combined_df[combined_df['percent_change_class'] == 'x']
combined_df_x_only.shape

(119049, 14)

In [143]:
# As the stock market is not open during the weekends and US holidays, the reddit posts that falls within the dates will not be able to determine the change in S&P500
print("The total number of days that post were being captured: {}".format(raw_combined_df['Date'].nunique()))
print("The number of non-trading days: {}".format(combined_df_x_only['Date'].nunique()))
print("The number of trading days between 2019 & 2020: {}".format((raw_combined_df['Date'].nunique() - combined_df_x_only['Date'].nunique())))

The total number of days that post were being captured: 729
The number of non-trading days: 227
The number of trading days between 2019 & 2020: 502


In [144]:
# 77% of the orginal data is retained after removing non-trading days
print("Percentage of post remaining after removing post from non-trading days: {}".format(round(1 - (119049/x), 2)))

Percentage of post remaining after removing post from non-trading days: 0.77


### Obtaining dataset based on trading days

In [167]:
# Slicing relevant data (For title on daily basis)
dailytitle_df = raw_combined_df[(combined_df['percent_change_class'] != 'x') & \
                            (combined_df['author'] != "AutoModerator")]

In [146]:
# Slicing relevant data (For combination of title and post for 2019)
titlepost2019_df = raw_combined_df[(combined_df['percent_change_class'] != 'x') & \
                                   (combined_df['year'] == 2019) & 
                                   (combined_df['author'] != "AutoModerator")]

## Combining Title on daily basis
---

In [168]:
title_dict = {}

for date in dailytitle_df['Date'].unique():
    temp_df = dailytitle_df[dailytitle_df['Date'] == date]
    title = temp_df['title']
    title_dict[date] = " ".join(title)

In [169]:
dailytitle_df = pd.DataFrame(title_dict, index=[0]).T
dailytitle_df.head()

,0
2019-01-03,Backtesting moving average crossover Can someo...
2019-01-04,What’s your best performing stock today? How t...
2019-01-07,Come Join Quantum Stock Trading! Help reach fi...
2019-01-08,"Greenspan says stock market ""is still a bit to..."
2019-01-09,"ONC, ONCY &amp; ONC.WT What is everyone's opin..."


In [170]:
dailytitle_df = dailytitle_df.reset_index()
dailytitle_df.columns = ['Date', 'combined_title']
dailytitle_df.head()

,Date,combined_title
0,2019-01-03,Backtesting moving average crossover Can someo...
1,2019-01-04,What’s your best performing stock today? How t...
2,2019-01-07,Come Join Quantum Stock Trading! Help reach fi...
3,2019-01-08,"Greenspan says stock market ""is still a bit to..."
4,2019-01-09,"ONC, ONCY &amp; ONC.WT What is everyone's opin..."


In [171]:
start = datetime.now()
change_dailytitle_df = match_dates_and_pull_y(dailytitle_df, target)
end = datetime.now()
print(end - start)

0:00:00.276050


In [172]:
dailytitle_df = dailytitle_df.merge(change_dailytitle_df, left_index=True, right_index=True)
dailytitle_df.rename(columns = {0: 'percent_change_class'}, inplace = True)
dailytitle_df.head()

,Date,combined_title,percent_change_class
0,2019-01-03,Backtesting moving average crossover Can someo...,up
1,2019-01-04,What’s your best performing stock today? How t...,up
2,2019-01-07,Come Join Quantum Stock Trading! Help reach fi...,up
3,2019-01-08,"Greenspan says stock market ""is still a bit to...",up
4,2019-01-09,"ONC, ONCY &amp; ONC.WT What is everyone's opin...",up


In [175]:
print(dailytitle_df['percent_change_class'].value_counts())
print()
x = dailytitle_df['percent_change_class'].value_counts().sum()
print('Total number of inputs: {}'.format(x))

up      293
down    209
Name: percent_change_class, dtype: int64

Total number of inputs: 502


In [176]:
dailytitle_df.to_csv("data/dailytitle_df_uncleaned", index=False)

### Preprocessing of Text

In [188]:
def preprocess_title(df_posts):
    
    # Filling in null values
    df_posts.fillna(value = 'notext', inplace=True)

    #Obtaining length of title and text
    df = df_posts
    df['title_len'] = [len(x) for x in df['combined_title'].str.split(' ')]
    #df['text_len'] = [len(x) for x in df['selftext'].str.split(' ')]

    # removing html within post
    regex_html = r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'
    df['combined_title'].replace(regex=True,inplace=True, to_replace=regex_html,value=r'')
    #df['selftext'].replace(regex=True,inplace=True, to_replace=regex_html,value=r'')
    
    return df

In [190]:
dailytitle_df = preprocess_title(dailytitle_df)

In [191]:
#remove url and digits
def review_to_words(raw_review):
    
    # 2. Remove non-letters.
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review)
    letters_oneline = re.sub('([\r\n]+)',' ', letters_only) 
    letters = re.sub('([ ]{2,})',' ',letters_oneline)
    
    # 3. Convert to lower case, split into individual words.
    words = letters.lower().split()
    
    # 4. Remove stopwords.
    stops = set(stopwords.words('english'))
    stops.update(['stocks', 'notext'])
    data = [w for w in words if w not in stops]
    
    data = " ".join(data)
    
    #5. Tokenizing
    tokenizer = TweetTokenizer()
    meaningful_words = tokenizer.tokenize(data)
    
    # 6.Lemmatizing.
    lemmatizer = WordNetLemmatizer()
    tokens_lem = [lemmatizer.lemmatize(word) for word in meaningful_words]
    data = [w for w in tokens_lem if w not in stops]
    
    # 7. Join the words back into one string separated by space, and return the result.
    return(" ".join(data))

In [192]:
dailytitle_df['title_cleaned'] = dailytitle_df['combined_title'].map(review_to_words)

In [193]:
dailytitle_df.head()

,Date,combined_title,percent_change_class,title_len,title_cleaned
0,2019-01-03,Backtesting moving average crossover Can someo...,up,108,backtesting moving average crossover someone e...
1,2019-01-04,What’s your best performing stock today? How t...,up,503,best performing stock today explain option new...
2,2019-01-07,Come Join Quantum Stock Trading! Help reach fi...,up,209,come join quantum stock trading help reach fin...
3,2019-01-08,"Greenspan says stock market ""is still a bit to...",up,541,greenspan say stock market still bit high appl...
4,2019-01-09,"ONC, ONCY &amp; ONC.WT What is everyone's opin...",up,472,onc oncy amp onc wt everyone opinion company d...


In [194]:
dailytitle_df.isnull().sum()

Date                    0
combined_title          0
percent_change_class    0
title_len               0
title_cleaned           0
dtype: int64

In [195]:
dailytitle_df.to_csv("data/project_df_dailytitle_cleaned.csv", index=False)

## Combined Title + Post for 2019
---

In [177]:
titlepost2019_df.head()

,date,title,selftext,is_self,upvotes,n_comments,permalink,author,month,year,Date,year_and_month,week_of_year,percent_change_class
0,2019-01-03 18:02:40,Backtesting moving average crossover,"Hello guys, \nI was trying to backtest a movi...",1.0,1.0,0.0,/r/stocks/comments/ac4c0u/backtesting_moving_a...,bhandarimohit2029,1,2019,2019-01-03,2019-01,1,up
2,2019-01-03 20:12:03,Can someone ELI5 what a cross signal Index is ...,I just finished Margin call on Netflix and was...,1.0,1.0,0.0,/r/stocks/comments/ac55o3/can_someone_eli5_wha...,tellmetheworld,1,2019,2019-01-03,2019-01,1,up
3,2019-01-03 20:54:22,Your AM Global Stocks Preview and a whole lot ...,\n\n### US Stocks\n\n* **US stocks index futu...,1.0,1.0,16.0,/r/stocks/comments/ac5gf6/your_am_global_stock...,QuantalyticsResearch,1,2019,2019-01-03,2019-01,1,up
4,2019-01-03 21:01:23,Gld outperforms snp and Dow,Gld has outperformed the snp and Dow since 200...,1.0,1.0,30.0,/r/stocks/comments/ac5ibv/gld_outperforms_snp_...,1anon2y3mous,1,2019,2019-01-03,2019-01,1,up
5,2019-01-03 21:17:40,Alternative to Yahoo Finance,"Hi there, i switched from Android to iOS.\n\nI...",1.0,1.0,10.0,/r/stocks/comments/ac5mqv/alternative_to_yahoo...,h0ly88,1,2019,2019-01-03,2019-01,1,up


In [183]:
titlepost2019_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67482 entries, 0 to 507461
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  67482 non-null  datetime64[ns]
 1   title                 67482 non-null  object        
 2   selftext              46810 non-null  object        
 3   is_self               67482 non-null  float64       
 4   upvotes               67482 non-null  float64       
 5   n_comments            67482 non-null  float64       
 6   permalink             67482 non-null  object        
 7   author                67482 non-null  object        
 8   month                 67482 non-null  int64         
 9   year                  67482 non-null  int64         
 10  Date                  67482 non-null  datetime64[ns]
 11  year_and_month        67482 non-null  object        
 12  week_of_year          67482 non-null  UInt32        
 13  percent_change_

### Preprocessing of text

In [196]:
def preprocess(df_posts):
    
    # Filling in null values
    df_posts.fillna(value = 'notext', inplace=True)

    #Obtaining length of title and text
    df = df_posts
    df['title_len'] = [len(x) for x in df['title'].str.split(' ')]
    df['text_len'] = [len(x) for x in df['selftext'].str.split(' ')]

    # removing html within post
    regex_html = r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'
    df['title'].replace(regex=True,inplace=True, to_replace=regex_html,value=r'')
    df['selftext'].replace(regex=True,inplace=True, to_replace=regex_html,value=r'')
    
    return df

In [197]:
titlepost2019_df = preprocess(titlepost2019_df)

In [200]:
#remove url and digits
def review_to_words(raw_review):
    
    # 2. Remove non-letters.
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review)
    letters_oneline = re.sub('([\r\n]+)',' ', letters_only) 
    letters = re.sub('([ ]{2,})',' ',letters_oneline)
    
    # 3. Convert to lower case, split into individual words.
    words = letters.lower().split()
    
    # 4. Remove stopwords.
    stops = set(stopwords.words('english'))
    stops.update(['notext', 'sma', 'year', 'month', 'day', 'gt', 'click', 'long', 'term', 'stock', 'market', 'way', \
                  'global', 'idea', 'ty', 'amp', 'ta', 'cap', 'st', 'live', 'pre', 'company', 'like', 'time', \
                  'price', 'earnings', 'removed', 'option', 'trading', 'trade', 'today', 'want', 'sell', 'thought', \
                  'investing', 'invest', 'money', 'good', 'new', 'week', 'think', 'going', 'account', 'option', \
                  'stocks', 'know', 'make', 'shares', 'spy', 'million', 'buying', 'fund', 'revenue', ' short', \
                  'investement', 'looking', 'buy', 'investment', 'people', 'share', 'china', 'guy', 'right', 'curr', \
                  'key', 'avg', 'fn', 'tn', 'tp', 'bb', 'rsi', 'hang', 'seng', 'chart', 'sch', 'stofu', 'yr', 'feel', \
                  'morning', 'coo', 'elon', 'mask', 'wall', 'street', 'stoxx', 'usd', 'calendar', 'kong', 'united', \
                  'macd', 'quote', 'thanks', 'advance', 'information', 'technology', 'tl', 'dr', 'dae', 'growth', 'news', \
                  'investor', 'dividend', 'billion', 'roth', 'ira', 'etf', 'question', 'index', 'hong', 'state', 'fy', 'fp',\
                  'rate', 'cut', 'goldman', 'sachs', 'economic', 'appreciated', 'reached', 'sector', 'dt', 'tgt', 'warren', \
                  'real', 'estate', 'basis', 'say', 'discretionary', 'yahoo', 'finance', 'morgan', 'stanley', 'south', \
                  'korea', 'oversold', 'according', 'com', 'briefing', 'president', 'captial', 'student', 'buffet', \
                  'td', 'ameritrade', 'yield', 'charles', 'schwab', 'quarter', 'reported', 'high', 'short', 'portfolio', \
                  'robinhood', 'donald', 'trump', 'bln', 'federal', 'reserve', 'bank', 'currently', 'robin', 'hood',  \
                  'cash', 'changed', 'webp', 'dow', 'jones', 'dir', 'wti', 'capital', 'large', 'hit', 'beat', 'map', \
                  'credit', 'card', 'expected', 'neutral', 'rated', 'exp', 'dax', 'glf', 'finished', 'cac', 'euro', \
                  'format', 'png', 'auto', 'nikkei', 'yesterday', 'technical', 'shanghai','need', 'said', 'thing', 'work', \
                  'balance', 'sheet', 'estimate', 'interactive', 'broker', 'overbought', 'low', 'bond', 'future', 'risk', \
                  'position', 'advice', 'little', 'strategy', 'read', 'higher', 'apple', 'symbol', 'ago', 'tesla', 'pay', \
                  'holding', 'average', 'cnbc', 'margin', 'th', 'product', 'small', 'data', 'open', 'total', 'app', 'world',\
                  'profit', 'look', 'start', 'gain', ' future', 'lot', 'loss', 'let', 'free', 'post', \
                  'big', 'business', 'return', 'sale', 'tax', 'shit', 'financial', 'report', 'best', 'bought', \
                  'ratio', 'closing', 'closed', 'value', 'really', 'help', 'hold', 'come', 'end', 'got', 'play', \
                  'service', 'dollar', 'better', 'worth', 'trying', 'recession', 'thinking', 'gold', 'getting', 'debt', \
                  'mini', 'nasdaq', 'oz', 'link', 'afternoon', 'ipo', 'bbl', 'insider', 'lyft', 'ba', 'yes', 'health', 'care', \
                  'profit', 'use', 'point', 'deal', 'cost', 'le', ' selling', 'recently', 'understand', \
                  'japanese', 'release', 'nd', 'mean', 'number', 'tomorrow', 'wondering', 'actually', 'drop', 'platform',\
                  'current', 'selling', 'plan', 'sure', 'great', 'world', 'industry', 'fed', 'using', 'income', 'potential',\
                  'signal', 'result', 'txn', 'io', 'past', 'close', 'making', 'order', 'bad', 'fuck', 'analyst', 'opinion',\
                  'fee', 'based', 'pretty', 'started', 'dd', 'bit', 'monday', 'tuesday', 'wednesday', ' thursday', 'friday', \
                  'sold', 'perf', 'economy', 'job', 'lower', 'coming', 'research', 'hi', 'fucking', 'chinese', 'google', 'reason', \
                  'old'])
    
    data = [w for w in words if w not in stops]
    
    data = " ".join(data)
    
    #5. Tokenizing
    tokenizer = TweetTokenizer()
    meaningful_words = tokenizer.tokenize(data)
    
    # 6.Lemmatizing.
    lemmatizer = WordNetLemmatizer()
    tokens_lem = [lemmatizer.lemmatize(word) for word in meaningful_words]
    data = [w for w in tokens_lem if w not in stops]
    
    # 7. Join the words back into one string separated by space, and return the result.
    return(" ".join(data))

In [199]:
titlepost2019_df['first_title_cleaned'] = titlepost2019_df['title'].map(review_to_words)
titlepost2019_df['first_selftext_cleaned'] = titlepost2019_df['selftext'].map(review_to_words)

In [201]:
# update the function by updating the stopwords to run again. 
titlepost2019_df['final_title_cleaned'] = titlepost2019_df['title'].map(review_to_words)
titlepost2019_df['final_selftext_cleaned'] = titlepost2019_df['selftext'].map(review_to_words)

### Combinging Title and Post into a single column

In [205]:
titlepost2019_df['fresh_meat'] = titlepost2019_df['first_title_cleaned'] + " " + titlepost2019_df['first_selftext_cleaned']
titlepost2019_df['lean_meat'] = titlepost2019_df['final_title_cleaned'] + " " + titlepost2019_df['final_selftext_cleaned']
titlepost2019_df['lean_meat_len'] = [len(x) for x in titlepost2019_df['lean_meat'].str.split(' ')]
titlepost2019_df.dropna(inplace=True)

In [206]:
titlepost2019_df.head()

,date,title,selftext,is_self,upvotes,n_comments,permalink,author,month,year,Date,year_and_month,week_of_year,percent_change_class,title_len,text_len,first_title_cleaned,first_selftext_cleaned,final_title_cleaned,final_selftext_cleaned,fresh_meat,lean_meat,lean_meat_len
0,2019-01-03 18:02:40,Backtesting moving average crossover,"Hello guys, \nI was trying to backtest a movi...",1.0,1.0,0.0,/r/stocks/comments/ac4c0u/backtesting_moving_a...,bhandarimohit2029,1,2019,2019-01-03,2019-01,1,up,4,212,backtesting moving average crossover,hello guy trying backtest moving average cross...,backtesting moving crossover,hello backtest moving crossover indian found i...,backtesting moving average crossover hello guy...,backtesting moving crossover hello backtest mo...,85
2,2019-01-03 20:12:03,Can someone ELI5 what a cross signal Index is ...,I just finished Margin call on Netflix and was...,1.0,1.0,0.0,/r/stocks/comments/ac55o3/can_someone_eli5_wha...,tellmetheworld,1,2019,2019-01-03,2019-01,1,up,16,59,someone eli cross signal index used trading,finished margin call netflix intrigued using i...,someone eli cross used,call netflix intrigued indicator sort proof sk...,someone eli cross signal index used trading fi...,someone eli cross used call netflix intrigued ...,24
3,2019-01-03 20:54:22,Your AM Global Stocks Preview and a whole lot ...,\n\n### US Stocks\n\n* **US stocks index futu...,1.0,1.0,16.0,/r/stocks/comments/ac5gf6/your_am_global_stock...,QuantalyticsResearch,1,2019,2019-01-03,2019-01,1,up,24,1033,global stock preview whole lot news need read ...,u stock u stock index future dropping sharply ...,preview whole negative guidance aapl spook,u u dropping sharply front p negative guidance...,global stock preview whole lot news need read ...,preview whole negative guidance aapl spook u u...,327
4,2019-01-03 21:01:23,Gld outperforms snp and Dow,Gld has outperformed the snp and Dow since 200...,1.0,1.0,30.0,/r/stocks/comments/ac5ibv/gld_outperforms_snp_...,1anon2y3mous,1,2019,2019-01-03,2019-01,1,up,5,20,gld outperforms snp dow,gld outperformed snp dow since gold bad rap in...,gld outperforms snp,gld outperformed snp since rap,gld outperforms snp dow gld outperformed snp d...,gld outperforms snp gld outperformed snp since...,8
5,2019-01-03 21:17:40,Alternative to Yahoo Finance,"Hi there, i switched from Android to iOS.\n\nI...",1.0,1.0,10.0,/r/stocks/comments/ac5mqv/alternative_to_yahoo...,h0ly88,1,2019,2019-01-03,2019-01,1,up,4,65,alternative yahoo finance,hi switched android io always used mystocks ap...,alternative,switched android always used mystocks android ...,alternative yahoo finance hi switched android ...,alternative switched android always used mysto...,23


In [208]:
titlepost2019_df.percent_change_class.value_counts()

up      39987
down    27495
Name: percent_change_class, dtype: int64

In [211]:
titlepost2019_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67482 entries, 0 to 507461
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    67482 non-null  datetime64[ns]
 1   title                   67482 non-null  object        
 2   selftext                67482 non-null  object        
 3   is_self                 67482 non-null  float64       
 4   upvotes                 67482 non-null  float64       
 5   n_comments              67482 non-null  float64       
 6   permalink               67482 non-null  object        
 7   author                  67482 non-null  object        
 8   month                   67482 non-null  int64         
 9   year                    67482 non-null  int64         
 10  Date                    67482 non-null  datetime64[ns]
 11  year_and_month          67482 non-null  object        
 12  week_of_year            67482 non-null  UInt3

In [210]:
titlepost2019_df.isnull().sum().sum()

0

In [212]:
titlepost2019_df.to_csv("data/project_titlepost2019_df_cleaned.csv", index=False)